In [1]:
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import requests
import csv
import urllib.request
import urllib.parse
import bs4
import googlemaps
import pandas as pd
from selenium import webdriver

driver_loc = "../externlib/chromedriver/chromedriver.exe"

options = webdriver.ChromeOptions()

options.add_argument("window-size=1920x1080")

driver = webdriver.Chrome(driver_loc, options=options)

baseUrl = "https://www.diningcode.com/"

driver.get(baseUrl)

driver.current_url

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/66536062.py:22: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_loc, options=options)


'https://www.diningcode.com/'

In [2]:
time.sleep(10)
# 팝업창 종료
closePath = '//*[@id="div_popup"]/div/div[3]'
driver.find_element_by_xpath(closePath).click()

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/3936362319.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(closePath).click()


In [3]:
# 검색 하기
searchPath = '//*[@id="txt_keyword"]'
driver.find_element_by_xpath(searchPath).send_keys("강남구 신사동")

searchPath = '//*[@id="t_search"]/button'
driver.find_element_by_xpath(searchPath).click()

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/3427237415.py:3: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).send_keys("강남구 신사동")
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/3427237415.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(searchPath).click()


In [4]:
# 결과 더보기 클릭
time.sleep(2)
arrowBtn = '//*[@id="root"]/div/div/div[2]/div[3]/div'
driver.find_element_by_xpath(arrowBtn).click()

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/3476121079.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowBtn).click()


In [5]:
# 20대 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[1]/span
# 30대 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[2]/span
# 40대 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[3]/span
# 50대 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[4]/span
# 60대 이상 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[5]/span

# 여성 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[2]/li[1]/span
# 남성 //*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[2]/li[2]/span

In [10]:
# 20대
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[1]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/342871320.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/342871320.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/342871320.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/342871320.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()


In [11]:
# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/4210789060.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()


,업소명,별점,리뷰수,좋아요수,소재지지번
0,다운타우너 청담점,4.5,66,255,서울특별시 강남구 신사동 666-25
1,카츠 바이 콘반,4.9,21,99,서울특별시 강남구 신사동 647-22
2,쮸즈,4.1,67,697,서울특별시 강남구 신사동 540-4
3,압구정 하루,4.2,32,135,서울시 강남구 신사동 663-9
4,미미면가 2호점,3.9,72,463,서울시 강남구 신사동 512-21 지하1층
...,...,...,...,...,...
84,카페파이브바,4.6,5,1,서울특별시 강남구 신사동 662-17
85,닭으로 가,4.1,14,68,서울시 강남구 신사동 662-6
86,미토우,5.0,1,12,서울특별시 강남구 신사동 563-18
87,호화반점,4.5,12,22,서울특별시 강남구 신사동 661-4


In [12]:
fileName = '../dataset/30대.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

In [ ]:
# 30대
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[2]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

In [ ]:
# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

In [ ]:
fileName = '../dataset/30대.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

In [13]:
# 40대
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[3]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/40대.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/1106596356.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/1106596356.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/1106596356.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/1106596356.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/11065963

In [14]:
# 50대
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[4]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/50대.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2310921092.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2310921092.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2310921092.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2310921092.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/23109210

In [15]:
# 60대 이상
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[1]/li[5]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/60대 이상.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/722286433.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/722286433.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/722286433.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/722286433.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/722286433.py

In [16]:
# 여성
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[2]/li[1]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/여성.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2729807031.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2729807031.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2729807031.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/2729807031.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/27298070

In [17]:
# 남성
# 체크박스 선택(for문 사용해야됨)
time.sleep(2)
featuresBtn = '//*[@id="root"]/div/div/div[1]/div[2]/div[1]/div[2]/div/ul[2]/li[2]/span'
driver.find_element_by_xpath(featuresBtn).click()

# 20곳 더보기 반복 선택(4번)
for i in range(0,4):
    moreInfo = '//*[@id="map"]/button[2]'
    driver.find_element_by_xpath(moreInfo).click()
# while !moreInfo:

html = driver.page_source
## BeautifulSoup로 페이지 소스 파싱
bs = bs4.BeautifulSoup(html, "html.parser")

bs

cafeName = []
foodScore = []
reviews = []
likes = []
address = []

for j in range(2,200):
    try :
        driver.switch_to.window(driver.window_handles[0])
        # 1번부터 차례대로 들어가기(for문)
        clikckInfo = '//*[@id="root"]/div/div/div[1]/div[4]/ol/div[num]'
        clikckInfoNum = clikckInfo.replace('num',str(j))
        clikckInfoNum

        driver.find_element_by_xpath(clikckInfoNum).click()

        time.sleep(2)

        driver.switch_to.window(driver.window_handles[1])
        currentUrl = driver.current_url

        driver.get(currentUrl)
        html = driver.page_source
        ## BeautifulSoup로 페이지 소스 파싱
        bs = bs4.BeautifulSoup(html, "html.parser")

        bs
        time.sleep(2)

        # 식당명 추출
        cafeCheck = bs.find(attrs={"class":"tit-point"} ).text.split("\n")
        cafeCheck

        time.sleep(2)

        # cafeName = []
        for i in range(0,len(cafeCheck)) :
            if cafeCheck[i] == '':
                pass
            else :
                cafeName.append(cafeCheck[i])
        cafeName

        # 별점 추출
        # foodScore = []
        foodScoreCheck = bs.findAll(attrs={"id":"lbl_review_point"} )[0].text.split("점")
        foodScore.append(foodScoreCheck[0])
        foodScore

        # 리뷰 수 추출
        # reviews = []
        reviewsCheck = bs.find(attrs={"class":"point"} ).text
        reviewsCheck

        # 연속 숫자 정보 추출
        reviews.append(re.findall(r'\d+', reviewsCheck)[0])
        reviews

        # 좋아요 숫자 추출
        # likes = []
        likeCheck = bs.find(attrs={"class":"favor button"} ).text
        likeCheck
        # 연속 숫자 정보 추출
        likes.append(re.findall(r'\d+', likeCheck)[0])
        likes

        # 주소 추출 locat
        # address = []
        addressCheck = bs.find(attrs={"class":"locat"} ).text
        address.append(addressCheck)
        address

        cafeDf = pd.DataFrame(cafeName, columns=["업소명"])

        scoreDf = pd.DataFrame(foodScore, columns=["별점"])

        reviewDF = pd.DataFrame(reviews, columns=["리뷰수"])

        likeDf = pd.DataFrame(likes, columns=["좋아요수"])

        addressDf = pd.DataFrame(address, columns=["소재지지번"])

        # 현재 실행중인 1개 창 종료
        time.sleep(2)
        driver.close()

        driver.switch_to.window(driver.window_handles[0])
        arrowInfo = '//*[@id="arrow"]'
        driver.find_element_by_xpath(arrowInfo).click()

    except :
        if len(driver.window_handles) >= 2 :
            driver.close()
        else:
            driver.switch_to.window(driver.window_handles[0])

# 체크박스 해제
driver.find_element_by_xpath(featuresBtn).click()

resultNum = pd.concat([cafeDf,scoreDf,reviewDF,likeDf,addressDf], axis=1)
resultNum

fileName = '../dataset/남성.csv'
resultNum.to_csv(fileName, index=False, encoding="utf-8-sig")

C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/678689510.py:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(featuresBtn).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/678689510.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(moreInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/678689510.py:33: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(clikckInfoNum).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/678689510.py:107: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath(arrowInfo).click()
C:\Users\SMART18\AppData\Local\Temp/ipykernel_1772/678689510.py